In [ ]:
pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Load the CSV files
nse_df = pd.read_csv('/content/nse.csv')
stocks_df = pd.read_csv('/content/stocks_with_sectors.csv')

# Normalize column names
nse_df.columns = nse_df.columns.str.strip().str.upper()
stocks_df.columns = stocks_df.columns.str.strip().str.upper()

# Clean relevant columns and convert to lowercase for case-insensitive matching
nse_df['SYMBOL'] = nse_df['SYMBOL'].astype(str).str.strip().str.lower()
nse_df['NAME OF COMPANY'] = nse_df['NAME OF COMPANY'].str.strip().str.lower()
stocks_df['STOCKS'] = stocks_df['STOCKS'].str.strip().str.lower()

# Function to clean company names by removing common stop words
def clean_name(name):
    stop_words = ["limited", "inc", "ltd", "pvt", "company", "&", "and", "corp", "corporation", "group", "the"]
    for word in stop_words:
        name = name.replace(word, '').strip()
    return name

# Clean the names in the DataFrames
nse_df['CLEANED NAME'] = nse_df['NAME OF COMPANY'].apply(clean_name)
stocks_df['CLEANED STOCK'] = stocks_df['STOCKS'].apply(clean_name)

# Initialize lists for results
symbols = []
names = []
unmatched_stocks = []

# Manual mapping for specific stocks that require attention
manual_mapping = {
    "ballarpur inds.": ("BIL", "BALLARPUR INDUSTRIES"),
    "akanksha power&infra": ("AKANKSHA", "AKANKSHA POWER & INFRASTRUCTURE"),
    "zeal aqua": ("ZEAL", "ZEAL AQUA"),
    "paushak": ("PAUSHAK", "PAUSHAK LIMITED"),
    "genpharmasec": ("GENPHARMA", "GENPHARMA SECURITIES"),
    "goodricke group": ("GOODRICKE", "GOODRICKE GROUP"),
    "rdb rasayans": ("RDB", "RDB RASAYANS"),
}

# Step 1: Enhanced matching logic
for index, row in stocks_df.iterrows():
    stock_name = row['CLEANED STOCK']

    # Check for manual mapping first
    if stock_name in manual_mapping:
        symbol, name = manual_mapping[stock_name]
        symbols.append(symbol)
        names.append(name)
        continue

    # Try to match by exact symbol
    matched_row = nse_df[nse_df['SYMBOL'] == row['STOCKS']]

    # If no match by symbol, try to match by cleaned name
    if matched_row.empty:
        matched_row = nse_df[nse_df['CLEANED NAME'] == stock_name]

    # If still unmatched, try substring matching
    if matched_row.empty:
        matched_row = nse_df[nse_df['CLEANED NAME'].str.contains(stock_name, na=False, regex=False)]

    # If there's still no match, check for variations by splitting the stock name and checking each part
    if matched_row.empty:
        stock_name_parts = stock_name.split()
        for part in stock_name_parts:
            matched_row = nse_df[nse_df['CLEANED NAME'].str.contains(part, na=False, regex=False)]
            if not matched_row.empty:
                break

    # If still unmatched, use fuzzy matching for remaining unmatched stocks
    if matched_row.empty:
        match = process.extractOne(stock_name, nse_df['NAME OF COMPANY'], score_cutoff=70)
        if match:
            matched_name = match[0]
            matched_row = nse_df[nse_df['NAME OF COMPANY'] == matched_name]

    # If still unmatched, try fuzzy matching on cleaned names with additional logic for specific cases
    if matched_row.empty:
        match = process.extractOne(stock_name, nse_df['CLEANED NAME'], score_cutoff=70)
        if match:
            matched_name = match[0]
            matched_row = nse_df[nse_df['CLEANED NAME'] == matched_name]

    # Additional check for specific stocks with slight variations
    if matched_row.empty:
        if stock_name == "akanksha power&infra":
            alternative_names = ["akanksha power and infrastructure", "akanksha power & infra"]
            for alt_name in alternative_names:
                match = process.extractOne(alt_name, nse_df['NAME OF COMPANY'], score_cutoff=70)
                if match:
                    matched_name = match[0]
                    matched_row = nse_df[nse_df['NAME OF COMPANY'] == matched_name]
                    break

    if matched_row.empty:
        if stock_name == "goodricke group":
            alternative_names = ["goodricke", "goodricke ltd"]
            for alt_name in alternative_names:
                match = process.extractOne(alt_name, nse_df['NAME OF COMPANY'], score_cutoff=70)
                if match:
                    matched_name = match[0]
                    matched_row = nse_df[nse_df['NAME OF COMPANY'] == matched_name]
                    break

    # Append match results
    if not matched_row.empty:
        symbols.append(matched_row.iloc[0]['SYMBOL'])
        names.append(matched_row.iloc[0]['NAME OF COMPANY'])
    else:
        symbols.append(None)
        names.append(None)
        unmatched_stocks.append(row['STOCKS'])

# Update stocks_df with match results
stocks_df['SYMBOL'] = symbols
stocks_df['NAME OF COMPANY'] = names

# Step 2: Review unmatched stocks
if unmatched_stocks:
    print(f"Unmatched stocks ({len(unmatched_stocks)}):")
    for stock in unmatched_stocks:
        print(f"Stock: {stock} requires manual review.")
else:
    print("All stocks matched successfully.")

# Step 3: Ensure the final output preserves the original stock order
final_columns = ['MAJOR SECTOR', 'STOCKS', 'WEIGHT', 'SYMBOL', 'NAME OF COMPANY']
result_df = stocks_df[final_columns]

# Step 4: Save the final DataFrame to a CSV file
result_df.to_csv('/content/mapped_stocks_with_sectors.csv', index=False)

# Final completion message
if not unmatched_stocks:
    print("Matching complete. Data has been saved as 'mapped_stocks_with_sectors.csv'.")
else:
    print("Some stocks require manual review. Please check the above list.")


Unmatched stocks (2):
Stock: akanksha power&infra requires manual review.
Stock: goodricke group requires manual review.
Some stocks require manual review. Please check the above list.
